In [410]:
# https://www.kaggle.com/datasets/aiaiaidavid/the-big-dataset-of-ultra-marathon-running/discussion/420633

In [412]:
# import libraries

In [413]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt

In [414]:
df = pd.read_csv('runners.csv')

C:\Users\annas\AppData\Local\Temp\ipykernel_2116\3092615535.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('runners.csv')


In [415]:
#see the data that has been imported

In [416]:
df.head()

,Year of event,Event dates,Event name,Event distance/length,Event number of finishers,Athlete performance,Athlete club,Athlete country,Athlete year of birth,Athlete gender,Athlete age category,Athlete average speed,Athlete ID
0,2018,06.01.2018,Selva Costera (CHI),50km,22,4:51:39 h,Tnfrc,CHI,1978.0,M,M35,10.286,0
1,2018,06.01.2018,Selva Costera (CHI),50km,22,5:15:45 h,Roberto Echeverría,CHI,1981.0,M,M35,9.501,1
2,2018,06.01.2018,Selva Costera (CHI),50km,22,5:16:44 h,Puro Trail Osorno,CHI,1987.0,M,M23,9.472,2
3,2018,06.01.2018,Selva Costera (CHI),50km,22,5:34:13 h,Columbia,ARG,1976.0,M,M40,8.976,3
4,2018,06.01.2018,Selva Costera (CHI),50km,22,5:54:14 h,Baguales Trail,CHI,1992.0,M,M23,8.469,4


In [417]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7461195 entries, 0 to 7461194
Data columns (total 13 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Year of event              int64  
 1   Event dates                object 
 2   Event name                 object 
 3   Event distance/length      object 
 4   Event number of finishers  int64  
 5   Athlete performance        object 
 6   Athlete club               object 
 7   Athlete country            object 
 8   Athlete year of birth      float64
 9   Athlete gender             object 
 10  Athlete age category       object 
 11  Athlete average speed      object 
 12  Athlete ID                 int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 740.0+ MB


In [418]:
df.shape

(7461195, 13)

In [419]:
#clean up data

In [420]:
#Only want Lithuanian Races

In [421]:
# Step 1 extract country data from column "Event name" into a new column "location"
df['location'] = df['Event name'].str.split('(').str[1].str.rstrip(')')

In [422]:
# Step 2 show only LTU races
df = df[df['location'] == 'LTU']

In [423]:
# Check what length/distance races took place
df['Event distance/length'].unique()

array(['6h', '82km', '100km', '65.8km', '83.7km', '50km', '47.4km',
       '67.9km', '47.6km', '70.7km', '21h', '101km'], dtype=object)

In [424]:
# Filter rows that contain 'km' and exclude rows with 'h'
df = df[df['Event distance/length'].str.contains('km', na=False)]

In [425]:
# Rename column Event distance/length to distance
df = df.rename(columns={'Event distance/length': 'distance'})

In [426]:
# Remove null values from 'Athlete year of birth' column
df = df.dropna(subset=['Athlete year of birth'])

In [427]:
# Remove LTU from event name
df['Event name'] = df['Event name'].str.split('(').str.get(0)

In [428]:
# Convert 'Event dates' column to datetime format
df['Event dates'] = pd.to_datetime(df['Event dates'], format='%d.%m.%Y')

In [429]:
# Convert 'Athlete year of birth' to numeric
df['Athlete year of birth'] = pd.to_numeric(df['Athlete year of birth'], errors='coerce')

In [430]:
# Determine athlete age
df['athlete_age'] = df['Year of event'] - df['Athlete year of birth']

In [431]:
#Clean up athlete age
df['athlete_age'] = df['Year of event'] - df['Athlete year of birth'].astype(int)

In [432]:
#Remove h from Athlete performance
df['Athlete performance'] = df['Athlete performance'].str.split(' ').str.get(0)

In [433]:
#Drop columns: Athlete Club, Athlete year of birth, Athlete age category, location
df = df.drop(['Athlete club', 'Athlete year of birth', 'Athlete age category', 'location'], axis = 1)

In [434]:
#Check for null values
df.isna().sum()

Year of event                0
Event dates                  0
Event name                   0
distance                     0
Event number of finishers    0
Athlete performance          0
Athlete country              0
Athlete gender               0
Athlete average speed        0
Athlete ID                   0
athlete_age                  0
dtype: int64

In [435]:
#Check for dupes
df[df.duplicated() == True]

,Year of event,Event dates,Event name,distance,Event number of finishers,Athlete performance,Athlete country,Athlete gender,Athlete average speed,Athlete ID,athlete_age


In [436]:
#Reset index
df = df.reset_index(drop = True)

In [437]:
#Fix types

In [438]:
df.dtypes

Year of event                         int64
Event dates                  datetime64[ns]
Event name                           object
distance                             object
Event number of finishers             int64
Athlete performance                  object
Athlete country                      object
Athlete gender                       object
Athlete average speed                object
Athlete ID                            int64
athlete_age                           int64
dtype: object

In [439]:
df['athlete_age'] = df['athlete_age'].astype(int)

In [440]:
df['Athlete average speed'] = df['Athlete average speed'].astype(float)

In [441]:
#Remove 'km' suffix in distance column and convert to string
df['distance'] = df['distance'].str.replace('km', '').astype(float)

In [442]:
# Round up the values in the 'distance' column
df['distance'] = np.ceil(df['distance'])

In [443]:
#Rename columns
df = df.rename(columns= {'Year of event' : 'year',
                         'Event dates' : 'race_day',
                         'Event name' : 'race_name',
                         'Event number of finishers' : 'race_number_of_finishers',
                         'Athlete performance' : 'athlete_performance',
                         'Athlete country' : 'athlete_country',
                         'Athlete gender' : 'gender',
                         'Athlete average speed' : 'average_speed',
                         'Athlete ID' : 'athlete_id',
                         'athlete_age' : 'age',
                        })

In [444]:
#Reorder columns
df = df[['year', 'race_day', 'race_name', 'distance', 'athlete_performance', 'athlete_country', 'gender', 'average_speed', 'age', 'race_number_of_finishers', 'athlete_id']]

In [445]:
#Get weather data for the races

In [446]:
#Get a list of race dates
df['race_day'].unique()

<DatetimeArray>
['2018-06-16 00:00:00', '2018-06-14 00:00:00', '2018-10-20 00:00:00',
 '2016-06-18 00:00:00', '2017-06-17 00:00:00', '2019-05-19 00:00:00',
 '2019-06-15 00:00:00', '2019-07-21 00:00:00', '2019-10-19 00:00:00',
 '2020-07-19 00:00:00', '2020-06-13 00:00:00', '2020-10-17 00:00:00',
 '2021-07-17 00:00:00', '2021-06-26 00:00:00', '2021-10-16 00:00:00',
 '2022-07-03 00:00:00', '2022-06-11 00:00:00', '2022-10-15 00:00:00',
 '2009-04-18 00:00:00', '2010-06-26 00:00:00', '2011-05-28 00:00:00',
 '2012-07-21 00:00:00', '2013-07-20 00:00:00']
Length: 23, dtype: datetime64[ns]

In [447]:
#Import weather data
#https://www.visualcrossing.com/weather/weather-data-services

In [448]:
weather_df = pd.read_csv('weather_data.csv')

In [ ]:
#See the data that has been imported
weather_df.info()

In [450]:
weather_df.head()

,datetime,tempmax,tempmin,temp,humidity,preciptype,windspeed,pressure,moonphase,icon
0,2009-04-18,10.1,0.1,5.8,53.2,NaN,31.3,1009.6,0.78,partly-cloudy-day
1,2010-06-26,23.0,17.0,19.6,82.9,NaN,24.0,1010.6,0.50,partly-cloudy-day
2,2011-05-28,17.0,11.5,14.9,80.0,rain,19.3,1012.4,0.85,rain
3,2012-07-21,18.0,11.5,14.8,84.8,rain,19.6,1012.9,0.08,rain
4,2013-07-20,18.0,10.0,13.8,78.6,rain,22.3,1012.3,0.42,rain


In [451]:
#Convert data type to datetime
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])

In [452]:
#Merge the DataFrames on 'race_day' and 'datetime'
df2 = pd.merge(df, weather_df, left_on='race_day', right_on='datetime', how='inner')

In [453]:
#Rename column 'icon'
df2.rename(columns={'icon': 'weather_description'}, inplace=True)

In [454]:
#Check for null values
df2.isnull().sum()

year                           0
race_day                       0
race_name                      0
distance                       0
athlete_performance            0
athlete_country                0
gender                         0
average_speed                  0
age                            0
race_number_of_finishers       0
athlete_id                     0
datetime                       0
tempmax                        0
tempmin                        0
temp                           0
humidity                       0
preciptype                  1011
windspeed                      0
pressure                       0
moonphase                      0
weather_description            0
dtype: int64

In [455]:
#Replace NaN values in the 'preciptype' column with 'no rain'
df2['preciptype'].fillna('no rain', inplace=True)

In [456]:
#Drop columns
df2.drop(columns=['tempmax', 'tempmin', 'pressure', 'moonphase'], inplace=True)

In [ ]:
#Quickly visualise the data

sns.pairplot(df2, kind='reg', plot_kws={'line_kws': {'color': 'red'}})

C:\Users\annas\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\annas\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\annas\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\annas\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating

In [ ]:
#Quickly check for correlations

#Select only numeric columns
numeric_df = df2.select_dtypes(include='number')

#Generate the correlation matrix
correlation_matrix = numeric_df.corr()

#Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
#Is long distance running more popular among men or women?
sns.histplot(df2['gender'])

In [ ]:
#What is age distribution?
sns.histplot(df2['age'], kde='True', bins = 45)

In [ ]:
#How old is the oldest runner?
df2['age'].max()

In [ ]:
#How old is the youngest runner?
df2['age'].min()

In [ ]:
#How old is the oldest female runner?
df2[df2['gender'] == 'F']['age'].max()

In [ ]:
#What is median runner age?
df2['age'].median()

In [ ]:
#Does average runner speed change with age?

#Define age bins and labels
age_bins = [10, 20, 30, 40, 50, 60, 70, 80]
age_labels = ['10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80']

#Create a new column 'age_group' with age categories
df2['age_group'] = pd.cut(df2['age'], bins=age_bins, labels=age_labels, right=False)

#Group by 'age_group' and 'gender', and calculate the median average speed
median_speed_per_age_group_gender = df2.groupby(['age_group', 'gender'])['average_speed'].median().reset_index()

#Create the line plot
plt.figure(figsize=(12, 8))
sns.lineplot(data=median_speed_per_age_group_gender, x='age_group', y='average_speed', hue='gender', marker='o', palette='viridis')

#Customize the plot
plt.title('Median Average Speed by Age Group and Gender')
plt.xlabel('Age Group')
plt.ylabel('Median Average Speed')
plt.legend(title='Gender')
plt.grid(True)

#Show the plot
plt.show()

In [ ]:
#Difference in speed for different distances male to female
df2.groupby(['distance', 'gender'])['average_speed'].mean()

In [ ]:
#Average speed for different distances male to female visualisation

#Calculate the average speed by distance and gender
avg_speed_df = df2.groupby(['distance', 'gender'])['average_speed'].mean().reset_index()

# Define custom color palette
custom_palette = {'M': 'blue', 'F': 'pink'}

#Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(data=avg_speed_df, x='distance', y='average_speed', hue='gender', palette=custom_palette)

#Customize the plot
plt.title('Average Speed by Distance and Gender')
plt.xlabel('Distance (km)')
plt.ylabel('Average Speed')
plt.legend(title='Gender')
plt.grid(True)

#Show the plot
plt.show()

In [ ]:
#Violinplot for average speed for different distances male to female
sns.violinplot(data = df, x = 'distance', y = 'average_speed', hue = 'gender', split = True, inner = 'quart')

In [ ]:
#Does long-distance running become more popular?

#Create the plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=df2, x='year', y='race_number_of_finishers', marker='o', color='blue')

#Customize the plot
plt.title('Number of Finishers per Year')
plt.xlabel('Year')
plt.ylabel('Number of Finishers')
plt.grid(True)

#Show the plot
plt.show()

In [ ]:
#Is there a correlation between age group and race distance?

#Create a scatter plot with a regression line
plt.figure(figsize=(10, 6))
sns.regplot(data=df2, x='age', y='distance', scatter_kws={'s': 50}, line_kws={'color': 'red'})

#Customize the plot
plt.title('Correlation between Age and Race Distance')
plt.xlabel('Age')
plt.ylabel('Race Distance (km)')
plt.grid(True)

#Show the plot
plt.show()

In [ ]:
#Does temperature or humidity affect the average speed of racers?

#Set up the plotting area
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

#Scatter plot for temperature vs. average speed
sns.regplot(data=df2, x='temp', y='average_speed', scatter_kws={'s': 50}, line_kws={'color': 'red'}, ax=axes[0])
axes[0].set_title('Average Speed vs. Temperature')
axes[0].set_xlabel('Temperature (°C)')
axes[0].set_ylabel('Average Speed (km/h)')
axes[0].grid(True)

#Scatter plot for humidity vs. average speed
sns.regplot(data=df2, x='humidity', y='average_speed', scatter_kws={'s': 50}, line_kws={'color': 'blue'}, ax=axes[1])
axes[1].set_title('Average Speed vs. Humidity')
axes[1].set_xlabel('Humidity (%)')
axes[1].set_ylabel('Average Speed (km/h)')
axes[1].grid(True)

#Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
#Does average speed decline with age?

sns.lmplot(data = df, x = 'age', y = 'average_speed', hue = 'gender', scatter_kws={'s': 10} )

In [ ]:
#Does percipitation affect the male vs female ration of racers?

# Aggregate the data
gender_distribution = df2.groupby(['preciptype', 'gender']).size().unstack(fill_value=0)

# Calculate male-to-female ratio
gender_distribution['ratio'] = gender_distribution['M'] / gender_distribution['F']
print(gender_distribution)

In [ ]:
#Does percipitation affect the male vs female ration of racers? visualisation

#Aggregate the data
gender_distribution = df2.groupby(['preciptype', 'gender']).size().reset_index(name='count')

#Create a bar plot
plt.figure(figsize=(10, 6))
sns.barplot(data=gender_distribution, x='preciptype', y='count', hue='gender', palette='viridis')

#Customize the plot
plt.title('Gender Distribution by Precipitation Type')
plt.xlabel('Precipitation Type')
plt.ylabel('Number of Racers')
plt.legend(title='Gender')
plt.grid(True)

#Show the plot
plt.show()